# OpenStreetMap data wrangling, SQL design and analysis

In [3]:
# Import required modules
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import csv
import codecs
import cerberus
from time import time
import sqlite3
from pprint import pprint

In [3]:
# Defone function to count the types of nodes 
def count_tags(filename):
    iter_obj = ET.iterparse(filename)
    d = defaultdict(int)
    for event, element in iter_obj:
        d[element.tag] += 1
    return d

In [4]:
# Count all types of nodes 
t0=time()
print count_tags('kyiv_ukraine.osm')
print time()-t0

defaultdict(<type 'int'>, {'node': 1401056, 'nd': 1721147, 'bounds': 1, 'member': 79530, 'tag': 618817, 'relation': 7350, 'way': 212674, 'osm': 1})
14.8539249897


In [2]:
# Count 'tags' in 'relations'
t0 = time()
counter = 0 
for _, element in ET.iterparse('kyiv_ukraine.osm'):
    if element.tag == 'relation':
        for tag in element.iter('tag'):
            counter += 1
print time() - t0

13.4803678989


In [2]:
# counting phone entries in 'nodes' 'tags'
t0 = time()
counter = 0 
for event, element in ET.iterparse('kyiv_ukraine.osm'):
    if element.tag == 'node':
        for tag in element.iter('tag'):
            if tag.attrib['k'] == 'phone' or tag.attrib['k'] == 'contact:phone':
                counter += 1
print time() - t0

14.8219199181


In [3]:
counter

1474

In [4]:
# Count phone entries in 'ways' 'tags'
t0 = time()
counter = 0 
for _, element in ET.iterparse('kyiv_ukraine.osm'):
    if element.tag == 'way':
        for tag in element.iter('tag'):
            if tag.attrib['k'] == 'phone' or tag.attrib['k'] == 'contact:phone':
                counter += 1
print time() - t0

16.6546189785


In [5]:
counter

305

In [9]:
# Audit tag key string types
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        k_value = element.attrib['k']
        if lower.search(k_value):
            keys['lower'] += 1
        elif lower_colon.search(k_value):
            keys['lower_colon'] += 1
        elif problemchars.search(k_value):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [10]:
t0=time()
print process_map('kyiv_ukraine.osm')
print time()-t0

{'problemchars': 0, 'lower': 463963, 'other': 717, 'lower_colon': 154137}
16.4316270351


In [19]:
# Extract unique user ids
def get_user(element):
    if element.tag == 'way' or element.tag == 'node':
        if 'uid' in element.attrib.keys():
            uid = element.attrib['uid']
        else:
            uid = 'uid_absent'
    else:
        uid = None
    return uid 


def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))
    return users

In [20]:
t0=time()
user_ids = process_map('kyiv_ukraine.osm')
print time()-t0

17.741451025


In [21]:
print 'type:', type(user_ids)
print 'lenght:', len(user_ids)
missing_uids = 0
index_list = []
for i in user_ids:
    if i == 'uid_absent':
        missing_uids += 1
        index_list.append('uid_missing')
    else:
        index_list.append('uid_present')
print 'missing uids:', missing_uids
#print 'missing uid index', index_list.index('uid_missing')

type: <type 'set'>
lenght: 2288
missing uids: 1


In [22]:
# Extract unique user names
def get_user(element):
    if element.tag == 'way' or element.tag == 'node':
        if 'user' in element.attrib.keys():
            user = element.attrib['user']
        else:
            user = 'user_name_absent'
    else:
        user = None
    return user

def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        if get_user(element):
            users.add(get_user(element))
    return users

In [23]:
t0=time()
user_names = process_map('kyiv_ukraine.osm')
print time()-t0

18.4127700329


In [24]:
print 'type:', type(user_names)
print 'lenght:', len(user_names)
missing_user_names = 0 
for i in user_names:
    if i == 'user_name_absent':
        missing_user_names += 1
print 'missing user names:', missing_user_names

type: <type 'set'>
lenght: 2292
missing user names: 1


In [17]:
# Extract unique tag k strings
def get_k(element):
    if element.tag == 'tag':
        k = element.attrib['k']
    else:
        k = None
    return k


def process_map(filename):
    ks = set()
    for _, element in ET.iterparse(filename):
        if get_k(element):
            ks.add(get_k(element))
    return ks

In [18]:
t0 = time()
unique_k_strings = process_map('kyiv_ukraine.osm')
print time() - t0

15.8906619549


In [19]:
len(unique_k_strings)

1153

In [20]:
# Define function to classify unique key strings into the three types: 
# lower case, lower case with colon, problematic and other
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_string_type(string, keys, lowers, lowers_colon, problematics, others):
    if lower.search(string):
        keys['lower'] += 1
        lowers.append(string)
    elif lower_colon.search(string):
        keys['lower_colon'] += 1
        lowers_colon.append(string)
    elif problemchars.search(string):
        keys['problemchars'] += 1
        problematics.append(string)
    else:
        keys['other'] += 1
        others.append(string)
    return keys, problematics, others

In [21]:
keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
lowers = []
lowers_colon = []
problematics = []
others = []

for string in unique_k_strings:
    key_string_type(string, keys, lowers, lowers_colon, problematics, others)
    
keys

{'lower': 480, 'lower_colon': 540, 'other': 133, 'problemchars': 0}

In [22]:
sorted(lowers)

['_extrude_',
 '_tessellate_',
 '_visibility_',
 'abandoned',
 'access',
 'admin_level',
 'administrative',
 'advertising',
 'aerialway',
 'aeroway',
 'agricultural',
 'aircraft',
 'allotments',
 'alt_name',
 'amenity',
 'animal',
 'animal_boarding',
 'animal_keeping',
 'architect',
 'area',
 'artist_name',
 'artwork_type',
 'atm',
 'attraction',
 'author',
 'automated',
 'backcountry',
 'backrest',
 'bank',
 'barbed_tape',
 'barbed_wire',
 'barrier',
 'beauty',
 'beds',
 'bench',
 'bic',
 'bicycle',
 'bicycle_parking',
 'bicycle_road',
 'bin',
 'boat',
 'bollard',
 'books',
 'born',
 'boundary',
 'boutique',
 'branch',
 'brand',
 'breakfast',
 'brewery',
 'bridge',
 'building',
 'bunker_type',
 'bus',
 'bus_lines',
 'busway',
 'button_operated',
 'cables',
 'cafe',
 'canoe',
 'capacity',
 'capital',
 'car',
 'car_parts',
 'car_repair',
 'carpenter',
 'cash',
 'cash_in',
 'castle_type',
 'cemetery',
 'charge',
 'chemist',
 'childcare',
 'circuits',
 'circumference',
 'clinic',
 'closes

In [23]:
sorted(lowers_colon)

['abandoned:amenity',
 'abandoned:barrier',
 'abandoned:building',
 'abandoned:foutain',
 'abandoned:highway',
 'abandoned:historic',
 'abandoned:landuse',
 'abandoned:man_made',
 'abandoned:power',
 'abandoned:public_transport',
 'abandoned:railway',
 'abandoned:shop',
 'access:conditional',
 'addr:city',
 'addr:country',
 'addr:district',
 'addr:door',
 'addr:flats',
 'addr:floor',
 'addr:housename',
 'addr:housenumber',
 'addr:interpolation',
 'addr:office',
 'addr:officenumber',
 'addr:place',
 'addr:postcode',
 'addr:region',
 'addr:street',
 'addr:suburb',
 'addr:unit',
 'aerialway:access',
 'aerodrome:type',
 'alt_name:de',
 'alt_name:en',
 'alt_name:eo',
 'alt_name:es',
 'alt_name:ru',
 'alt_name:uk',
 'alt_name:vi',
 'animal_keeping:type',
 'area:highway',
 'barrier:barbed_tape',
 'barrier:barbed_wire',
 'barrier:keypad',
 'branch:ru',
 'brand:en',
 'brand:ru',
 'brand:uk',
 'bridge:structure',
 'building:apartments',
 'building:cladding',
 'building:color',
 'building:colour'

In [24]:
sorted(others)

['CEMT',
 'ISO3166-1',
 'ISO3166-1:alpha2',
 'ISO3166-1:alpha3',
 'ISO3166-1:numeric',
 'ISO3166-2',
 '_Name_',
 'addr:city:en',
 'addr:housename:de',
 'addr:housenumber_1',
 'addr:interpolation_1',
 'addr:street:en',
 'addr:street_1',
 'amenity_1',
 'amenity_2',
 'artwork:3D',
 'associatedStreet',
 'bridge_1',
 'building:levels:underground',
 'building:parts:horizontal',
 'building:parts:vertical',
 'building:roof:shape',
 'building_1',
 'bunker_type_1',
 'cinema:3D',
 'compartment:NW',
 'currency:EUR',
 'currency:RUB',
 'currency:RUR',
 'currency:UAH',
 'currency:USD',
 'demolished:piste:type',
 'depot:vehicle:tram',
 'description_1',
 'destination:lang:en:backward',
 'destination:lang:en:lanes',
 'destroyed:name:ru',
 'destroyed:name:uk',
 'disused:name:ru',
 'disused:name:uk',
 'drink_1',
 'e-road:class',
 'fixme:building:part',
 'fixme:building:parts',
 'fuel:GTL_diesel',
 'fuel:HGV_diesel',
 'fuel:e10',
 'fuel:e85',
 'fuel:octane_100',
 'fuel:octane_80',
 'fuel:octane_91',
 'fuel

In [6]:
# Find values listed under 'addr:street:en' and 'addr:street_1'
t0 = time()
streets_under_alt_k = []
for _, element in ET.iterparse('kyiv_ukraine.osm'):
    if element.tag == 'tag':
        if element.attrib['k'] == 'addr:street:en' or element.attrib['k'] == 'addr:street_1':
            streets_under_alt_k.append(element.attrib['v'])
print time() - t0
for i in streets_under_alt_k:
    print i

15.197660923
Petra Hryhorenka Avenue
Saksahanskoho Street
Kiltseva Road
15 км
Фрунзе
Petra Hryhorenka Avenue
Revutskogo
Saksahanskoho
Dragomanova
Ирпенская улица


In [2]:
'''Auditing phone numbers'''

'''def audit_phone_number(phone_number)'''

#regex_normal = re.compile(r'(\+?38)?\W*(0?\W*\d{2})?\W*(\d\W*\d\W*\d)\W*(\d{2})\W*(\d{2})')
#regex_normal = re.compile(r'(\+?38)?\W*(0?\d{2})?\W*(\d\d\d)\W*(\d{2})\W*(\d{2})')
#regex_normal = re.compile(r'(38)?\W*(0)?\W*(\d{2})?\W*(\d{3})\W*(\d{2})\W*(\d{2})')
#regex_38 = re.compile(r'(\+?)(38)\W*(0)\W*(\d{2})\W*(\d{3})\W*(\d{2})\W*(\d{2})')
regex_38 = re.compile(r'(3)\W*(8)\W*(0)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\b')
#regex_0xx = re.compile(r'^\W?(0\d{2})\W*(\d{3})\W*(\d{2})\W*(\d{2})\b')
regex_0xx = re.compile(r'^\W?(0)\W*(\d{2})\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\b')
#regex_800 = re.compile(r'\d\W800\W\d{3}\W\d{3}')
regex_800 = re.compile(r'^(0|8)\W*(800)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\W*(\d)\b')
regex_xxx = re.compile(r'^(\d{3})\W*(\d{2})\W*(\d{2})$')

In [3]:
ph = r'0800500-609'
m = regex_800.findall(ph)

In [4]:
m

[('0', '800', '5', '0', '0', '6', '0', '9')]

In [5]:
# Count regex matches 
t0 = time()
counter = {'w38':0, 'w800':0, 'w0xx':0, 'wxxx':0, 'problematic':0}
w38 = []
w800 = []
w0xx = []
wxxx = []
problematics = []
for event, elem in ET.iterparse('kyiv_ukraine.osm'):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if tag.attrib['k'] == "phone" or tag.attrib['k'] == "contact:phone":
                if regex_38.search(tag.attrib['v']):
                    counter['w38'] += 1
                    w38.append(tag.attrib['v'])
                elif regex_800.search(tag.attrib['v']):
                    counter['w800'] += 1
                    w800.append(tag.attrib['v'])
                elif regex_0xx.search(tag.attrib['v']):
                    counter['w0xx'] += 1
                    w0xx.append(tag.attrib['v'])
                elif regex_xxx.search(tag.attrib['v']):
                    counter['wxxx'] += 1
                    wxxx.append(tag.attrib['v'])
                else:
                    counter['problematic'] += 1
                    problematics.append(tag.attrib['v'])
print time() - t0

17.2634048462


In [6]:
print counter

{'wxxx': 21, 'w38': 1393, 'problematic': 36, 'w0xx': 292, 'w800': 37}


In [7]:
sorted(problematics)

['(44)4247431',
 '+1 347 868 0740',
 '+3-044-257-20-97',
 '+3-8-044-446-77e-70',
 '+30 (44) 536-99-06; +30 (44) 536-99-08; +30 (44) 536-99-07',
 '+3044 401-42-94',
 '+30442556013',
 '+38 44 2784864',
 '+38 44 425 03 98',
 '+380 (044) 235-73-82',
 '+380 (044) 275-33-00',
 '+380 (044) 360 02 09',
 '+380 (044) 486-18-08',
 '+380 (067) 912-20-66',
 '+380 44 01010',
 '+380 9905577327',
 '+380(044) 528-30-47',
 '+380-044-4172526',
 '+3800675055958',
 '+3804118875',
 '+38986073213',
 '+39 044 5939575',
 '+800 1800 1800',
 '044526',
 '08005005000',
 '102',
 '234-55-83;234-05-88;235-23-21',
 '287-32-11 066-563-57-29',
 '2870711,2870020',
 '2876149,2876216',
 '4-60-85',
 '5-74-41',
 '67 401 21 66, 044 287 5252',
 '8097-331-17-93',
 '88003000500',
 u'\u0420\u0435\u0433\u0438\u0441\u0442\u0440\u0430\u0442\u0443\u0440\u0430 - (044) 408-03-41, \u0412\u044b\u0437\u043e\u0432 \u0432\u0440\u0430\u0447\u0430 - (044) 408-74-40, \u041d\u0435\u043e\u0442\u043b\u043e\u0436\u043d\u0430\u044f \u043f\u043e\u04

In [8]:
phone_mapping = {
    '(44)4247431' : '+38-044-424-7431',
     '+1 347 868 0740' : '+1-347-868-0740', 
     '+3-044-257-20-97' : '+38-044-257-2097',
     '+3-8-044-446-77e-70' : '+38-044-446-7770',
     '+30 (44) 536-99-06; +30 (44) 536-99-08; +30 (44) 536-99-07' : '+38-044-536-9906', 
     '+3044 401-42-94' : '+38-044-401-4294',
     '+30442556013' : '+38-044-255-6013', 
     '+38 44 2784864' : '+38-044-278-4864', 
     '+38 44 425 03 98' : '+38-044-4250-0398', 
     '+380 (044) 235-73-82' : '+38-044-235-7382', 
     '+380 (044) 275-33-00' : '+38-044-275-3300', 
     '+380 (044) 360 02 09' : '+38-044-360 0209', 
     '+380 (044) 486-18-08' : '+38-044-486-1808', 
     '+380 (067) 912-20-66' : '+38 067 912-2066', 
     '+380 44 01010' : 'ERRONEUS', 
     '+380 9905577327' : '+38-099-055-77327', 
     '+380(044) 528-30-47' : '+38-044-528-3047',
     '+380-044-4172526' : '+38-044-417-2526', 
     '+3800675055958' : '+38-067-505-5958', 
     '+3804118875' : 'ERRONEOUS', 
     '+38986073213' : '+38-098-607-3213',
     '+39 044 5939575' : '+38-044-593-9575',
     '+800 1800 1800' : 'ERRONEUOUS', 
     '044526' : 'ERRONEOUS', 
     '08005005000' : 'ERRONEOUS', 
     '102' : ' ERRONEOUS', 
     '234-55-83;234-05-88;235-23-21' : '+38-044-234-5583', 
     '287-32-11 066-563-57-29' : '38-044-287-3211', 
     '2870711,2870020' : '+38-044-287-0711', 
     '2876149,2876216' : '+38-044-287-6149', 
     '4-60-85' : 'ERRONEOUS', 
     '5-74-41' : 'ERRONEOUS',
     '67 401 21 66, 044 287 5252' : '+38-067-401-2166', 
     '8097-331-17-93' : '+38-097-331-1793', 
     '88003000500' : 'ERRONEOUS', 
 u'\u0420\u0435\u0433\u0438\u0441\u0442\u0440\u0430\u0442\u0443\u0440\u0430 - (044) 408-03-41, \u0412\u044b\u0437\u043e\u0432 \u0432\u0440\u0430\u0447\u0430 - (044) 408-74-40, \u041d\u0435\u043e\u0442\u043b\u043e\u0436\u043d\u0430\u044f \u043f\u043e\u043c\u043e\u0449\u044c - (044) 497-60-61'
: '38-044-408-0341'}

In [7]:
phone_mapping['67 401 21 66, 044 287 5252']

'+38-067-401-2166'

In [15]:
# Define function to standardize phone numbers
def standardize_phone(number, phone_mapping):
    stnd = ''
    if regex_38.search(number):
        m = regex_38.search(number)
        stnd = stnd + '+' + m.group(1) + m.group(2) + '-' + m.group(3) + m.group(4) + m.group(5) + '-' + m.group(6) + \
                m.group(7) + m.group(8) + '-' + m.group(9) + m.group(10) + m.group(11) + m.group(12)
    elif regex_800.search(number):
        m = regex_800.search(number)
        stnd = stnd + m.group(1) + '-' + m.group(2) + '-' + m.group(3) + m.group(4) + m.group(5) + '-' + m.group(6) + \
                m.group(7) + m.group(8)
    elif regex_0xx.search(number):
        m = regex_0xx.search(number)
        stnd = stnd + '+38-' + m.group(1) + m.group(2) + '-' + m.group(3) + m.group(4) + m.group(5) + '-' + m.group(6) + \
                m.group(7) + m.group(8) + m.group(9) 
    elif regex_xxx.search(number):
        m = regex_xxx.search(number)
        stnd = stnd + '+38-044-' + m.group(1) + '-' + m.group(2) + '-' + m.group(3)
    else:
        stnd = phone_mapping[number]
    return stnd

In [16]:
standardize_phone('0 (44) 52026 08', phone_mapping)

'+38-044-520-2608'

In [19]:
# Count phones number entries in 'node' and 'way' 'tags'
t0 = time()
counter = 0 
for event, elem in ET.iterparse('kyiv_ukraine.osm'):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if tag.attrib['k'] == "phone":
                counter += 1
print time() - t0

17.3097550869


In [20]:
counter

1544

In [18]:
# Test standardize function
t0 = time()
phone_list = [] 
for event, elem in ET.iterparse('kyiv_ukraine.osm'):
    if elem.tag == "node" or elem.tag == "way":
        for tag in elem.iter("tag"):
            if tag.attrib['k'] == "phone":
                phone_list.append(standardize_phone(tag.attrib['v'], phone_mapping))
print time() - t0

16.7216579914


In [21]:
len(phone_list)

1544

In [22]:
phone_list

['+38-044-501-3297',
 '0-800-501-505',
 '+38-093-977-5883',
 '+38-044-247-4314',
 '+38-044-287-2000',
 '+38-044-529-6193',
 '+38-044-254-3610',
 '+38-044-288-9775',
 '+38-044-285-5625',
 '+38-044-354-0535',
 '+38-044-257-9429',
 '+38-044-234-3380',
 '+38-068-850-6060',
 '+38-044-468-2298',
 '+38-044-285-6550',
 '+38-044-275-3688',
 '+38-044-585-2728',
 '+38-044-258-0554',
 '+38-044-287-1002',
 '+38-044-234-7423',
 '+38-044-289-2020',
 '+38-044-289-33-22',
 '+38-044-284-4011',
 '+38-044-486-6336',
 '+38-080-030-1707',
 '+38-044-257-5594',
 '+38-067-443-6565',
 '+38-044-206-0562',
 '+38-044-331-5781',
 '+38-044-567-9196',
 '+38-099-010-9993',
 '+38-044-257-8640',
 '+38-044-259-7672',
 '+38-044-531-9400',
 '+38-044-288-8727',
 'ERRONEOUS',
 '+38-044-464-4734',
 '+38-044-288-5225',
 '+38-044-567-6768',
 '+38-044-456-0243',
 '+38-044-569-19-09',
 '+38-044-569-19-09',
 '+38-044-500-5000',
 '+38-044-234-5678',
 '+38-044-461-8788',
 '+38-044-417-3190',
 '+38-044-287-0711',
 '+38-044-254-2024',

In [113]:
"""
Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""
street_type_re = re.compile(r'\b\S+\.?$', re.UNICODE | re.IGNORECASE)


# expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            # "Trail", "Parkway", "Commons"]
    
expected =['вулиця'.decode('utf-8'), 'бульвар'.decode('utf-8'), 'тупик'.decode('utf-8'), 'проїзд'.decode('utf-8'),
          'проспект'.decode('utf-8'), 'алея'.decode('utf-8'), 'шосе'.decode('utf-8'), 'набережна'.decode('utf-8'),
          'узвіз'.decode('utf-8'), 'дорога'.decode('utf-8'), 'провулок'.decode('utf-8'), 'площа'.decode('utf-8'),
          'шоссе'.decode('utf-8')]

# UPDATE THIS VARIABLE
street_mapping = { 'ул.'.decode('utf-8') : 'вулиця'.decode('utf-8'),
           'ул'.decode('utf-8') : 'вулиця'.decode('utf-8'),
           'пл.'.decode('utf-8') : 'площа'.decode('utf-8'), 
           'шоссе-2'.decode('utf-8') : 'шоссе'.decode('utf-8'),
           'улица'.decode('utf-8') : 'вулиця'.decode('utf-8')
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    # osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osmfile, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    # osm_file.close()
    return street_types


def update_name(name, street_mapping):
    m = street_type_re.search(name)
    if m and m.group() in street_mapping:
        name = re.sub(street_type_re, street_mapping[m.group()], name)
    return name

"""
for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            if name == "West Lexington St.":
                ass. better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                ass. better_name == "Baldwin Road"
"""

'\nfor st_type, ways in st_types.iteritems():\n        for name in ways:\n            better_name = update_name(name, mapping)\n            print name, "=>", better_name\n            if name == "West Lexington St.":\n                ass. better_name == "West Lexington Street"\n            if name == "Baldwin Rd.":\n                ass. better_name == "Baldwin Road"\n'

In [115]:
print update_name('Парковая ул.'.decode('utf-8'), street_mapping)

Парковая вулиця


In [12]:
t0 = time()
anomalous_street_names = audit('kyiv_ukraine.osm')
print time() - t0

17.513010025


In [13]:
for i in anomalous_street_names.keys():
    print i

Паширова
Ленінградська
Лесная)
Парковая
Багірова
М.Хвильового
Градинська
Заболотного
Дімітрова
Академічна
Лінія
8
ул.
прикордонна
Березова
Берковецкая
Золотоворітська
Северинівська
Карьерная
Лугова
Вітянська
Глушкова
Лучистая
Рудыковская
Сковороди
Набережная
Червоноармійська
Ленинградская
Дзержинського
Кільцева
Вовчка
Тіниста
Виставкова
Сталинграда
Лес
шлях
Паркова
гоголя
Колгоспний
Кошова
Сагайдачного
Озерна
Драйзера
Лисенка
шевченка
Г.Сковороди
Садова
Михайлівська
Бурки
Центральная(Ленина)
Стрільців
Окружная
Богатирська
пл.
городок
Набережно-Корчеватская
Героїв
пер
Комарова
Компресорна
мікрорайон
Грушевского
Осенняя
Gladkivska
ул
Лебединская
Покровська
Петлюри
Орача
вулиця/Петропавлівська
Лютнева
Антонова
Варшавська
Нахимова
Заміський
11
10
Электротехническая
шоссе-2
Ломоносова
ул.Дзержинского
Аркадия
лютнева
Шевченка
Яблунська
zhylianskaya
В.Непопа
В.Лобановского
Полевая
Народная
Лісовий
переулок
Київська
Озерная
Прикордонна
Артема
народів
Майдану
1
Дніпра
Бальзака
Київ-Чернігів
Лен

In [30]:
# Create a set of (unique) streets
t0 = time()
kyiv_streets = set()
for _, elem in ET.iterparse('kyiv_ukraine.osm'):
    if elem.tag == 'tag' and is_street_name(elem):
        kyiv_streets.add(elem.attrib['v'])
print time() - t0

13.465720892


In [31]:
len(kyiv_streets)

1646

In [33]:
for street in sorted(kyiv_streets):
    print street

1-а Лінія вулиця
1-а Піщана вулиця
1-го Травня вулиця
1-й Червоний провулок
10
10-а Лінія
10-а Лінія вулиця
11-а Лінія
11-а Лінія вулиця
12-а Лінія вулиця
12-а Садова вулиця
13-а Лінія вулиця
14-а Лінія вулиця
15-а Лінія вулиця
16-а Лінія вулиця
17-а Лінія вулиця
18-а Лінія вулиця
19-а Лінія вулиця
2-а Лінія вулиця
2-а Нова вулиця
2-а Піщана вулиця
2-й Маршака провулок
2-й Північний тупик
20-а Лінія вулиця
20-а Садова вулиця
21-а Лінія вулиця
22-а Лінія вулиця
23-я Лінія вулиця
23-я Лінія провулок
23-я Садова вулиця
24-а Лінія вулиця
25-а Лінія вулиця
255, Київська вулиця
26-а Лінія вулиця
27-а Лінія вулиця
28 км + 900 м. автодороги Київ-Чернігів
28-а Лінія вулиця
29-а Лінія вулиця
3-я Лінія вулиця
30-а Лінія вулиця
31-а Лінія вулиця
32-а Лінія вулиця
33-я Лінія вулиця
34-а Лінія вулиця
35-а Лінія вулиця
36-а Лінія вулиця
37-а Лінія вулиця
38-а Лінія вулиця
39-а Лінія вулиця
4-а Лінія вулиця
40 років Жовтня вулиця
40-а Лінія вулиця
42-а Садова вулиця
5-а Лінія вулиця
5-а Садова вулиця


In the section below we retrieve the data from OSM, clean up, put in CSV and then import into SQL.

In [108]:
OSM_PATH = "kyiv_ukraine.osm"

In [109]:
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'((\w|_)+):((\w|_)+:?.*)')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [110]:
import Schema
SCHEMA = Schema.schema

In [111]:
# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

In [122]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for i in node_attr_fields:
            if i in element.attrib.keys():
                node_attribs[i] = element.attrib[i]
            else:
                # creating a placeholder ID and user name for the one missing uid / user name entry 
                node_attribs[i] = 111111
        for j in element.iter('tag'):
            prob = PROBLEMCHARS.search(j.attrib['k'])
            colon = LOWER_COLON.search(j.attrib['k'])
            if prob:
                pass
            elif colon:
                d = {'id':element.attrib['id']}
                d['key'] = colon.group(3)
                d['type'] = colon.group(1)
                if j.attrib['k'] == 'addr:street':
                    # fixing street names per previously identified issues and correction mapping 
                    d['value'] = update_name(j.attrib['v'], street_mapping)
                elif j.attrib['k'] == 'contact:phone':
                    d['value'] = standardize_phone(j.attrib['v'], phone_mapping)
                else: 
                    d['value'] = j.attrib['v']
                tags.append(d)
            else:
                d = {'id':element.attrib['id']}
                d['key'] = j.attrib['k']
                d['type'] = default_tag_type
                if j.attrib['k'] == 'phone':
                    d['value'] = standardize_phone(j.attrib['v'], phone_mapping)
                else:
                    d['value'] = j.attrib['v']
                tags.append(d)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for i in way_attr_fields:
            way_attribs[i] = element.attrib[i]
        for j in element.iter('tag'):
            prob = PROBLEMCHARS.search(j.attrib['k'])
            colon = LOWER_COLON.search(j.attrib['k'])
            if prob:
                pass
            elif colon:
                d = {'id':element.attrib['id']}
                d['key'] = colon.group(3)
                d['type'] = colon.group(1)
                if j.attrib['k'] == 'addr:street':
                    d['value'] = update_name(j.attrib['v'], street_mapping)
                elif j.attrib['k'] == 'contact:phone':
                    d['value'] = standardize_phone(j.attrib['v'], phone_mapping)
                else: 
                    d['value'] = j.attrib['v']
                tags.append(d)
            else:
                d = {'id':element.attrib['id']}
                d['key'] = j.attrib['k']
                d['type'] = default_tag_type
                if j.attrib['k'] == 'phone':
                    d['value'] = standardize_phone(j.attrib['v'], phone_mapping)
                else:
                    d['value'] = j.attrib['v']
                tags.append(d)
        counter = 0 
        for k in element.iter('nd'):
            d = {'id':element.attrib['id']}
            d['node_id'] = k.attrib['ref']
            d['position'] = counter
            way_nodes.append(d)
            counter += 1
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# helper function 1
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

# helper function 2
def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )

# helper function 3
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

# main function
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

t0 = time()
if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)
print time() - t0  

68.5272610188


In [4]:
# Define tables for the SQL database
nodes = '''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);'''

nodes_tags = '''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);'''

ways = '''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);'''

ways_tags = '''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);'''

ways_nodes = '''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);'''

In [5]:
# Create the SQL database
conn = sqlite3.connect('P3_project_Kyiv_submission_2.db')

In [6]:
# Create tables
cursor = conn.cursor()
cursor.execute(nodes)
cursor.execute(nodes_tags)
cursor.execute(ways)
cursor.execute(ways_tags)
cursor.execute(ways_nodes)
conn.commit()

In [7]:
t0 = time()
# Populate the database tables with data from CSVs
# Read in the csv file as a dictionary and formatting the data as a list of tuples,
# then insert the formatted data in the database table 

# populating the nodes_tags table
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type']) for i in dr]
    
cursor.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

# populating the ways table
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cursor.executemany("INSERT INTO ways(id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

# populating the nodes table
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'].decode("utf-8"), i['uid'], i['version'], i['changeset'], i['timestamp']) 
             for i in dr]

cursor.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", 
                   to_db)
conn.commit()

# populating the ways_tags table
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'].decode("utf-8"), i['value'].decode("utf-8"), i['type']) for i in dr]
    
cursor.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
conn.commit()

# populating the ways_nodes table 
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]
    
cursor.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)
conn.commit()

'''
cursor.execute('SELECT * FROM nodes_tags')
all_rows = cursor.fetchall()
print('1):')
pprint(all_rows[:20])
'''
print time() - t0

29.0603051186


In [8]:
conn.close()

In [28]:
# Query the sql database
# Establish connection and cursor
conn = sqlite3.connect('P3_project_Kyiv.db')
cursor = conn.cursor()

In [21]:
# Count unique users by their IDs in both “nodes” and “ways” nodes
query = '''SELECT uid, user, COUNT(*) as num from (SELECT uid, user FROM nodes UNION ALL SELECT uid, user from ways) as united 
GROUP BY uid ORDER BY num DESC LIMIT 10;'''

cursor.execute(query)
all_rows = cursor.fetchall()
pprint(all_rows)

[(561414, u'Freeways_me', 160871),
 (1676637, u'Kilkenni', 120949),
 (371387, u'matvey_kiev_ua', 93157),
 (440812, u'dudka', 64077),
 (435936, u'Legioner', 49977),
 (648633, u'kulyk', 45594),
 (348674, u'Cabeleira', 43623),
 (188947, u'D_i_m', 35208),
 (353472, u'Barbos', 34878),
 (1872841, u'1gorok', 32352)]


In [15]:
# Count the total number of “ways” and “nodes” attributable to the top 50 users
query = '''SELECT sum(num) FROM (SELECT uid, COUNT(*) as num from (SELECT uid FROM nodes UNION ALL SELECT uid from ways) 
as united GROUP BY uid ORDER BY num DESC LIMIT 50) as topsomany;'''

cursor.execute(query)
all_rows = cursor.fetchall()
pprint(all_rows)

[(1209007,)]


In [11]:
# Retrieve all unique types of amenities 
query = '''SELECT DISTINCT value FROM (SELECT value FROM nodes_tags WHERE key == "amenity") 
            as allamenities ORDER BY value;'''

cursor.execute(query)
all_rows = cursor.fetchall()
pprint(all_rows)

[(u'agency',),
 (u'arts_centre',),
 (u'atm',),
 (u'bank',),
 (u'bar',),
 (u'bar; pub',),
 (u'bbq',),
 (u'beargarden',),
 (u'beauty',),
 (u'bench',),
 (u'bicycle_parking',),
 (u'bicycle_rental',),
 (u'bicycle_repair_station',),
 (u'biergarten',),
 (u'boat_rental',),
 (u'bookmaker',),
 (u'bureau_de_change',),
 (u'bus_station',),
 (u'cafe',),
 (u'car_rental',),
 (u'car_sharing',),
 (u'car_wash',),
 (u'casino',),
 (u'charging_station',),
 (u'childcare',),
 (u'cinema',),
 (u'clinic',),
 (u'clock',),
 (u'club',),
 (u'college',),
 (u'community_centre',),
 (u'compressed_air',),
 (u'computer',),
 (u'convenience',),
 (u'courthouse',),
 (u'coworking_space',),
 (u'dentist',),
 (u'doctors',),
 (u'drinking_water',),
 (u'education',),
 (u'educational',),
 (u'educational_institute',),
 (u'embassy',),
 (u'emergency_service',),
 (u'fast_food',),
 (u'fire_station',),
 (u'fitness_center',),
 (u'fitness_station',),
 (u'food_court',),
 (u'fountain',),
 (u'fuel',),
 (u'government',),
 (u'grave_yard',),
 (u'g

In [29]:
# Retrieve all arts centers 
query = 'SELECT * FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "arts_centre");'

cursor.execute(query)
all_rows = cursor.fetchall()
pprint(all_rows[:5])

[(306524501,
  u'name',
  u"\u0426\u0435\u043d\u0442\u0440 \u043f\u0430\u043c'\u044f\u0442\u043a\u043e\u0437\u043d\u0430\u0432\u0441\u0442\u0432\u0430",
  u'regular'),
 (306524501, u'amenity', u'arts_centre', u'regular'),
 (306524501,
  u'alt_name',
  u'\u0413\u0430\u043b\u043b\u0435\u0440\u0435\u044f \u041b\u0430\u0432\u0440\u0430',
  u'regular'),
 (742273893,
  u'name',
  u'\u041f\u0456\u043d\u0447\u0443\u043a \u0410\u0440\u0442 \u0426\u0435\u043d\u0442\u0440',
  u'regular'),
 (742273893, u'email', u'info@pinchukartcentre.org', u'regular')]


In [21]:
conn.close()

I decided to run the rest of my queries through my Linux terminal, as it readily prints the unicode characters and they appear correctly in cyrilic. I document below all the queries which I ran in Linux. The commentary around and results of these queries are in the submitted pdf file. 

SELECT id, value FROM nodes_tags WHERE key == "name" AND id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "arts_centre");

SELECT COUNT (*) FROM nodes_tags WHERE key == "amenity" and value == "atm";

SELECT DISTINCT value FROM nodes_tags WHERE key == "operator" AND id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "atm") ORDER BY value;

SELECT * FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "coworking_space");

SELECT id, value FROM nodes_tags WHERE key == "name" AND id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "coworking_space");

SELECT COUNT (*) FROM nodes_tags WHERE key == "amenity" and value == "restaurant";

SELECT * FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "restaurant");

SELECT id, value FROM nodes_tags WHERE key == "name" AND id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "restaurant");

SELECT value, count(*) as num FROM nodes_tags WHERE key == "cuisine" AND id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "restaurant") GROUP BY value ORDER BY num DESC limit 10;

SELECT * FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key == "amenity" and value == "restaurant") AND id IN (SELECT id FROM nodestags WHERE key == "cuisine" and value IN ("georgian", "грузинская", "грузинскаякухня"));

SELECT DISTINCT value FROM nodes_tags WHERE key == 'power';

SELECT COUNT(*) FROM nodes_tags WHERE key == "power" and value == "generator";

SELECT * FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key == "power" and value == "generator") AND key == "source";

SELECT * FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key == "power" and value == "substation");

SELECT * FROM nodes_tags WHERE id IN (SELECT id FROM nodes_tags WHERE key == "power" and value == "substation") and key == 'voltage';

SELECT * FROM ways_tags WHERE id IN (SELECT id FROM ways_tags WHERE key == "phone") LIMIT 100;

SELECT * FROM ways_tags WHERE id IN (SELECT id FROM ways_tags WHERE id == "4357024") LIMIT 100;

